In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

In [4]:
# Load data
y = pd.read_csv("df_tot.csv")
y["Data"] = pd.to_datetime(y["Data"]).dt.date
y.set_index("Data", inplace=True)
y 

,Ospiti,Vacanze,Viaggi,Gita,Google Maps,Traffico,Traffico Milano,Mare,Montagna,Lago Garda,...,champions league,Partita calcio,Previsioni del tempo,Pioggia,Neve,Meteo,Meteo Agrate,autostrada A4,Pizza,Ristorante
Data,,,,,,,,,,,,,,,,,,,,,
2021-06-01,319.0,6.125000,3.708333,3.500000,4.458333,2.000000,1.791667,16.208333,2.333333,0.000000,...,3.250000e+00,0.000000,1.250000,2.916667,1.750000,17.333333,2.458333,6.500000,0.000000,13.458333
2021-06-02,396.0,7.458333,5.660714,5.309524,3.386905,5.000000,1.148810,17.755952,3.976190,0.000000,...,2.107143e+00,0.000000,4.583333,2.511905,1.107143,19.904762,1.220238,7.690476,0.000000,16.458333
2021-06-03,188.0,7.296171,7.753378,3.229730,3.922297,3.576577,1.246622,18.762387,3.518018,0.000000,...,2.132883e+00,0.000000,3.934685,1.894144,1.619369,19.063063,3.575450,5.644144,0.000000,14.800676
2021-06-04,258.0,6.883095,6.715952,3.252381,4.179286,3.975238,1.110714,19.611190,4.105714,0.000000,...,3.166190e+00,0.289524,4.034762,4.767143,1.971905,22.666667,2.877381,4.875238,0.000000,14.538333
2021-06-05,571.0,6.716549,5.798389,2.486769,4.803404,2.972685,1.006135,18.208974,3.593043,0.000000,...,3.767286e+00,0.344857,3.886790,3.491357,2.513764,27.323944,5.621799,3.878148,0.000000,16.164693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,375.0,4.367012,2.657099,2.094199,11.293774,1.631865,2.807394,4.216981,0.028546,4.148074,...,4.444330e-21,3.063728,0.474055,2.775318,5.455785,16.288040,3.553792,1.075770,3.299307,17.058101
2022-11-26,732.0,3.275259,2.701158,1.570649,11.803664,1.223898,2.855545,4.016902,0.021409,4.111055,...,3.333248e-21,3.568629,0.355541,2.956489,5.414756,15.070197,3.029928,0.806827,2.474480,20.189409
2022-11-27,495.0,2.456444,2.702952,1.177987,10.633998,0.917924,2.933326,3.543927,0.016057,4.135375,...,2.499936e-21,3.270222,0.266656,2.779866,5.863150,14.313064,2.907862,0.605121,1.855860,20.735807


In [63]:
max_ospiti = y["Ospiti"].max()
min_ospiti = y["Ospiti"].min()
# print(max_ospiti, min_ospiti)
print("Max ospiti: ", max_ospiti)
print("Min ospiti: ", min_ospiti)

Max ospiti:  1183.0
Min ospiti:  117.0


In [7]:
# normalize data with MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(y)
scaled_data = scaler.transform(y)
scaled_data

array([[0.18949343, 0.31115667, 0.19814918, ..., 0.28428015, 0.        ,
        0.31517002],
       [0.26172608, 0.37892153, 0.30252871, ..., 0.33635378, 0.        ,
        0.38551563],
       [0.06660413, 0.37067986, 0.41440815, ..., 0.24684343, 0.        ,
        0.34664599],
       ...,
       [0.35459662, 0.12470727, 0.14439878, ..., 0.02642721, 0.07702593,
        0.48581611],
       [0.10225141, 0.09349591, 0.13723097, ..., 0.01980993, 0.05776945,
        0.46147375],
       [0.16135084, 0.65354288, 1.        , ..., 0.03352839, 0.18859167,
        0.50451536]])

In [14]:
scaled_data.shape

(546, 29)

In [16]:
tf_dataset = tf.data.Dataset.from_tensor_slices(scaled_data)

In [17]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
  windows = ds.window(window_size, shift=shift, stride=stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

  windows = windows.flat_map(sub_to_batch)
  return windows

In [19]:
ds = make_window_dataset(tf_dataset, window_size=37, shift = 1, stride=1)

for example in ds.take(2):
  print(example.numpy())
  print(example.numpy().shape)

[[0.18949343 0.31115667 0.19814918 ... 0.28428015 0.         0.31517002]
 [0.26172608 0.37892153 0.30252871 ... 0.33635378 0.         0.38551563]
 [0.06660413 0.37067986 0.41440815 ... 0.24684343 0.         0.34664599]
 ...
 [0.17354597 0.21785312 0.13378409 ... 0.04332143 0.09978955 0.24238072]
 [0.07879925 0.2583968  0.10030994 ... 0.07272404 0.07484137 0.18168157]
 [0.21763602 0.27158708 0.07520479 ... 0.05908867 0.05613058 0.13615805]]
(37, 29)
[[0.26172608 0.37892153 0.30252871 ... 0.33635378 0.         0.38551563]
 [0.06660413 0.37067986 0.41440815 ... 0.24684343 0.         0.34664599]
 [0.13227017 0.34968583 0.35894457 ... 0.21321005 0.         0.34049444]
 ...
 [0.07879925 0.2583968  0.10030994 ... 0.07272404 0.07484137 0.18168157]
 [0.21763602 0.27158708 0.07520479 ... 0.05908867 0.05613058 0.13615805]
 [0.26547842 0.39001127 0.05637619 ... 0.05615273 0.04209769 0.10201589]]
(37, 29)


In [20]:
def dense_7_step(batch):
  # Shift features and labels one step relative to each other.
  return batch[:-7], batch[-7:]

In [22]:
dense_labels_ds = ds.map(dense_7_step)

#create a tensor for holding the labels and features
all_inputs = []
all_labels = []

for inputs,labels in dense_labels_ds.take(2):
  print(inputs.numpy(), "=>", labels.numpy())
  print(inputs.numpy().shape, "=>", labels.numpy().shape)

for inputs,labels in dense_labels_ds:
  all_inputs.append(inputs)
  all_labels.append(labels)

[[0.18949343 0.31115667 0.19814918 0.18984274 0.217732   0.12251147
  0.06829551 0.43283889 0.16295159 0.         0.17668779 0.
  0.05728158 0.         0.         0.03743154 0.05247953 0.1204737
  0.6635603  0.21301718 0.         0.0601134  0.14319242 0.11738908
  0.4802552  0.09618721 0.28428015 0.         0.31517002]
 [0.26172608 0.37892153 0.30252871 0.28801188 0.16489446 0.30628902
  0.04375323 0.47430341 0.27768329 0.         0.12112904 0.
  0.15720134 0.         0.         0.10977376 0.07247173 0.2653844
  0.31276876 0.13811004 0.         0.22045139 0.12332035 0.07426441
  0.55151781 0.04733232 0.33635378 0.         0.38551563]
 [0.06660413 0.37067986 0.41440815 0.17518021 0.19129736 0.21909126
  0.04748739 0.50126828 0.24568604 0.         0.54075456 0.
  0.19091146 0.         0.         0.07164745 0.09624625 0.23227465
  0.27697855 0.13979714 0.         0.18925049 0.09299096 0.10862601
  0.52819161 0.14026834 0.24684343 0.         0.34664599]
 [0.13227017 0.34968583 0.35894457 0

In [26]:
X = tf.concat(all_inputs, axis=0)
X = tf.reshape(X, (-1, 30, 29))
print(X.shape)
X

(510, 30, 29)


<tf.Tensor: shape=(510, 30, 29), dtype=float64, numpy=
array([[[0.18949343, 0.31115667, 0.19814918, ..., 0.28428015,
         0.        , 0.31517002],
        [0.26172608, 0.37892153, 0.30252871, ..., 0.33635378,
         0.        , 0.38551563],
        [0.06660413, 0.37067986, 0.41440815, ..., 0.24684343,
         0.        , 0.34664599],
        ...,
        [0.15572233, 0.03532222, 0.15424373, ..., 0.13550981,
         0.02851231, 0.67344037],
        [0.18761726, 0.25574544, 0.17714327, ..., 0.12042729,
         0.14462738, 0.65527902],
        [0.1641651 , 0.48934636, 0.21637284, ..., 0.14180162,
         0.23645811, 0.64596933]],

       [[0.26172608, 0.37892153, 0.30252871, ..., 0.33635378,
         0.        , 0.38551563],
        [0.06660413, 0.37067986, 0.41440815, ..., 0.24684343,
         0.        , 0.34664599],
        [0.13227017, 0.34968583, 0.35894457, ..., 0.21321005,
         0.        , 0.34049444],
        ...,
        [0.18761726, 0.25574544, 0.17714327, ..., 0.1

In [33]:
Y = tf.concat(all_labels, axis=0)
Y = tf.reshape(Y, (-1, 7, 29))
Y = Y[:,:,0]
Y

<tf.Tensor: shape=(510, 7), dtype=float64, numpy=
array([[0.1575985 , 0.09474672, 0.34990619, ..., 0.17354597, 0.07879925,
        0.21763602],
       [0.09474672, 0.34990619, 0.29268293, ..., 0.07879925, 0.21763602,
        0.26547842],
       [0.34990619, 0.29268293, 0.17354597, ..., 0.21763602, 0.26547842,
        0.22326454],
       ...,
       [0.12382739, 0.19418386, 0.18574109, ..., 0.24202627, 0.57692308,
        0.35459662],
       [0.19418386, 0.18574109, 0.18574109, ..., 0.57692308, 0.35459662,
        0.10225141],
       [0.18574109, 0.18574109, 0.24202627, ..., 0.35459662, 0.10225141,
        0.16135084]])>

### Build a simple dense model

In [45]:
# build a forecasting model using the Keras Sequential API
model = keras.Sequential(
    [
        layers.Flatten(input_shape=(30, 29)),
        layers.Dense(870, activation="relu", name="layer0"),
        # add dropout to prevent overfitting
        layers.Dropout(0.2),
        layers.Dense(435, activation="relu", name="layer1"),
        layers.Dropout(0.2),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dropout(0.2),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dropout(0.2),
        layers.Dense(7, name="layer4"),
    ]
)

In [46]:
model.compile(optimizer='adam', loss="mse")

In [47]:
model.fit(x=X, y=Y ,epochs=500)

Epoch 1/500
16/16 [==============================] - 1s 19ms/step - loss: 0.0504
Epoch 2/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0308
Epoch 3/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0263
Epoch 4/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0242
Epoch 5/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0226
Epoch 6/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0208
Epoch 7/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0198
Epoch 8/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0196
Epoch 9/500
16/16 [==============================] - 0s 22ms/step - loss: 0.0177
Epoch 10/500
16/16 [==============================] - 0s 21ms/step - loss: 0.0168
Epoch 11/500
16/16 [==============================] - 0s 20ms/step - loss: 0.0156
Epoch 12/500
16/16 [==============================] - 0s 22ms/step - loss: 0.0144
Epoch 13/500
16/16 [=====

### Simulate a request for forecasting the next 7 days

In [48]:
y_test = y.iloc[-30:]
y_test

,Ospiti,Vacanze,Viaggi,Gita,Google Maps,Traffico,Traffico Milano,Mare,Montagna,Lago Garda,...,champions league,Partita calcio,Previsioni del tempo,Pioggia,Neve,Meteo,Meteo Agrate,autostrada A4,Pizza,Ristorante
Data,,,,,,,,,,,,,,,,,,,,,
2022-10-31,580.0,4.371816,5.333387,0.676958,3.480818,4.313177,0.071639,14.316661,0.432894,4.593606,...,5.905746e-18,2.969876,0.541593,2.561456,4.033734,21.069135,2.306861,1.451180,10.952556,22.651186
2022-11-01,489.0,4.268862,4.840040,0.955635,4.820613,3.234883,1.689146,13.287496,0.949670,4.955204,...,4.429309e-18,3.197407,0.656195,3.991092,4.605301,21.131852,3.310146,1.088385,9.484417,19.328389
2022-11-02,221.0,5.149563,4.150863,0.852143,6.177960,2.426162,2.610610,11.382289,3.191419,4.185153,...,3.321982e-18,2.481388,2.356729,4.305819,4.224809,20.244722,4.170109,0.816289,8.196646,16.173375
2022-11-03,285.0,4.258006,4.821481,1.284941,6.331387,1.819622,1.999624,9.786716,3.612314,4.420115,...,2.491486e-18,2.069375,2.048797,3.791864,5.126940,21.027292,3.523415,0.612217,6.762068,14.609198
2022-11-04,371.0,4.193504,4.564027,1.880372,6.758957,1.364716,2.728885,10.090037,4.802986,4.658836,...,1.868615e-18,1.666614,1.744931,4.062648,5.199372,21.395469,4.205062,0.459162,6.498634,13.665232
2022-11-05,701.0,3.145128,3.423020,2.035279,6.110884,1.023537,2.317497,11.442528,4.779323,4.973294,...,1.401461e-18,3.229127,1.517032,3.546986,7.014112,19.348685,3.549630,0.344372,7.321892,13.176007
2022-11-06,435.0,2.358846,2.567265,3.016043,5.624830,0.767653,2.102706,10.696479,4.334492,6.188304,...,1.051096e-18,3.067679,1.356524,2.951906,6.718917,17.469847,3.505972,0.258279,6.481003,12.829922
2022-11-07,206.0,1.769135,1.925449,2.522449,6.635289,0.575740,2.212446,10.064026,5.063369,6.276644,...,7.883219e-19,2.654926,1.350726,3.359763,6.310021,16.185719,3.202396,0.193709,5.517002,12.424525
2022-11-08,294.0,1.806851,1.994087,2.771837,6.196467,0.431805,1.659335,10.478020,3.797527,4.927483,...,5.912414e-19,2.021194,1.983045,3.529822,5.702516,16.859289,4.881797,1.041115,7.197751,14.548394


In [57]:
y_test_scaled = scaler.fit_transform(y_test)
y_test_scaled = y_test_scaled.reshape(1,30,29)
print(y_test_scaled.shape)
y_test_scaled

(1, 30, 29)


array([[[6.22580645e-01, 3.11863153e-01, 2.16155408e-01, 0.00000000e+00,
         0.00000000e+00, 7.11999741e-01, 0.00000000e+00, 7.78965950e-01,
         3.35364109e-02, 3.29071168e-01, 6.55252203e-01, 1.00000000e+00,
         5.43255116e-02, 3.75893444e-05, 7.04157178e-01, 3.52687481e-01,
         2.21513012e-02, 2.26704115e-01, 2.27147956e-02, 1.00000000e+00,
         2.34400863e-01, 6.66267565e-02, 2.95731992e-01, 4.60489743e-04,
         7.78439548e-01, 0.00000000e+00, 3.51100327e-01, 9.25449324e-01,
         6.69923595e-01],
        [4.75806452e-01, 3.03518395e-01, 1.87238994e-01, 6.50041233e-02,
         7.95594534e-02, 5.30770676e-01, 4.57781970e-01, 7.04548007e-01,
         7.47168718e-02, 3.71839803e-01, 6.55932478e-01, 9.41089987e-01,
         4.07398205e-02, 2.67443803e-05, 5.00999623e-01, 3.15217183e-01,
         1.57603926e-02, 1.00000000e+00, 2.29127397e-02, 7.49940458e-01,
         2.64666822e-01, 8.89789852e-02, 8.72932318e-01, 6.18523302e-02,
         7.85665749e-01, 

In [58]:
# dates to be predicted
dates = pd.date_range(start=y.index[-1],periods=8)[1:]
dates

DatetimeIndex(['2022-11-30', '2022-12-01', '2022-12-02', '2022-12-03',
               '2022-12-04', '2022-12-05', '2022-12-06'],
              dtype='datetime64[ns]', freq='D')

In [61]:
# predict the next 7 days
pred = model.predict(y_test_scaled)
print(pred.shape)
pred

(1, 7)


array([[0.17109013, 0.1953297 , 0.2862624 , 0.6402192 , 0.38504863,
        0.1499837 , 0.17663933]], dtype=float32)

In [64]:
# inverse transform the predicted values
pred_unscaled = pred*(max_ospiti - min_ospiti) + min_ospiti
pred_unscaled

array([[299.38208, 325.22144, 422.1557 , 799.4737 , 527.4618 , 276.88263,
        305.29755]], dtype=float32)

In [65]:
# construct results dataframe
results = pd.DataFrame(pred_unscaled[0], columns=["Ospiti"], index=dates)
results

,Ospiti
2022-11-30,299.382080
2022-12-01,325.221436
2022-12-02,422.155701
2022-12-03,799.473694
2022-12-04,527.461792
2022-12-05,276.882629
2022-12-06,305.297546


In [66]:
# save the model with pickle
import pickle
pickle.dump(model, open("models/my_dense_model_ex", 'wb'))

# save the scaler with pickle
pickle.dump(scaler, open("models/scaler_ex", 'wb'))


2022-12-17 16:03:56.514625: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://2066c8ab-5d96-4e23-9b53-f5cd21f7ccca/assets
